## Problem 4:

In [1]:
import numpy as np
import numpy.linalg as lin
import scipy.stats as sts
import scipy.integrate as intgr
import scipy.optimize as opt
import matplotlib
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
from matplotlib import cm

In [2]:
data = np.loadtxt('MacroSeries.txt', delimiter=',')
c = data[:,0]
k = data[:,1]
w = data[:,2]
r = data[:,3]

In [58]:
def get_zseries(w, k, alpha):
    denom = (1 - alpha) * (k ** alpha)
    return np.log(w / denom)

def data_moments(c, k, w, r, alpha, beta, rho, mu):
    z = get_zseries(w, k, alpha)
    z0 = np.roll(z, 1)
    z0[0] = mu
    c0 = np.roll(c, 1)
    
    mom1 = np.mean(z - rho * z0 - (1 - rho) * mu)
    mom2 = np.mean((z - rho * z0 - (1 - rho) * mu) * z0)
    mom3 = np.mean(beta * alpha * np.exp(z0 + 1) * (k ** (alpha - 1))
                   * (c0 / c) - 1)
    mom4= np.mean((beta * alpha * np.exp(z0 + 1) * (k ** (alpha - 1))
                   * (c0 / c) - 1) * w)
    
    moms_model = np.array([mom1, mom2, mom3, mom4])
    
    return moms_model

def err_vec(c, k, w, r, alpha, beta, rho, mu):
    moms_data = data_moments(c, k, w, r, alpha, beta, rho, mu)
    moms_model = np.zeros(4)
    
    err_vec = (moms_data - moms_model) #/ moms_data
    return err_vec

def criterion(params, *args):
    alpha, beta, rho, mu = params
    c, k, w, r, W = args
    err = err_vec(c, k, w, r, alpha, beta, rho, mu)
    crit_val = (err.T @ W) @ err
    return crit_val

alpha_init = 0.45
beta_init = 0.93
rho_init = 0.7
mu_init = 10
params_init = np.array([alpha_init, beta_init, rho_init, mu_init])

W_hat = np.eye(4)
args_gmm = (c, k, w, r, W_hat)

results = opt.minimize(criterion, params_init, args=(args_gmm),
                       method='L-BFGS-B',
                       bounds=((1e-10,1-1e-10),(1e-10,1-1e-10),
                               (1e-10-1,1-1e-10),(1e-10,None)))

alpha_gmm, beta_gmm, rho_gmm, mu_gmm = results.x
print("Converged?", results.success)
print("alpha_gmm =", alpha_gmm, "\nbeta_gmm =", beta_gmm,
      "\nrho_gmm =", rho_gmm, "\nmu_gmm =", mu_gmm)
print("Criterion value=", results.fun)

    

Converged? True
alpha_gmm = 0.305387281933 
beta_gmm = 0.605901746719 
rho_gmm = 0.700000000001 
mu_gmm = 6.66623279681
Criterion value= 316.627930873
